In [1]:
import os
import json
import pandas as pd
import numpy as np
import csv
import msgpack

import tools
from config import config as c

In [2]:
path_to_data = '/media/nazar/DATA/datasets/SQuAD/dev-v1.1.json'

In [3]:
data = json.load(open(path_to_data, 'r'))

In [4]:
data.keys()

dict_keys(['version', 'data'])

In [5]:
print('data len', len(data['data']))
print(data['data'][0].keys())

data len 48
dict_keys(['title', 'paragraphs'])


In [6]:
data['data'][0]['title']

'Super_Bowl_50'

In [7]:
print(data['data'][0]['paragraphs'][0].keys())
print('context:\n', data['data'][0]['paragraphs'][0]['context'])
print('qas:\n', data['data'][0]['paragraphs'][0]['qas'][29])

dict_keys(['context', 'qas'])
context:
 Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
qas:
 {'question': 'What 2015 NFL team one the AFC playoff?', 'answers': [{'text': 'Denver Broncos', 'answer_start': 177}, {'text': 'Denver Broncos', 'answer_start': 177}, {'text': 'De

In [8]:
data['data'][0]['paragraphs'][0]['qas'][0].keys()

dict_keys(['question', 'answers', 'id'])

## JSON STRUCTURE

In [9]:
"""
dict:
    'data':list of dict
        'title': 'Super_Bowl_50'
        'paragraphs':list of dict
            'context': text
            'qas':list of dict
                'question':'Which NFL team represented the AFC at Super Bowl 50?'
                'id':'56be4db0acb8001400a502ec'
                'answers':list of dict   len=3
                    'answer_start':177
                    'text':'Denver Broncos'
    'version'
"""

"\ndict:\n    'data':list of dict\n        'title': 'Super_Bowl_50'\n        'paragraphs':list of dict\n            'context': text\n            'qas':list of dict\n                'question':'Which NFL team represented the AFC at Super Bowl 50?'\n                'id':'56be4db0acb8001400a502ec'\n                'answers':list of dict   len=3\n                    'answer_start':177\n                    'text':'Denver Broncos'\n    'version'\n"

## test read_set

In [10]:
paragraphs = tools.read_set(c.path_to_test_data)

In [11]:
len(paragraphs)

2067

In [12]:
par = paragraphs[0]

In [13]:
par.context

'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'

In [14]:
par.context[:177]

'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion '

In [15]:
len(par.qas)

30

In [16]:
qa = par.qas[0]

In [17]:
qa.question

'Which NFL team represented the AFC at Super Bowl 50?'

In [18]:
a = qa.answers[0]

In [19]:
a.start_char

177

In [20]:
a.text

'Denver Broncos'

## test tokenize_paragraphs

In [21]:
tokenized_paragraphs = tools.tokenize_paragraphs(paragraphs, c)

In [22]:
len(tokenized_paragraphs)

2067

In [23]:
par = tokenized_paragraphs[0]

In [24]:
par.context

['Super',
 'Bowl',
 '50',
 'was',
 'an',
 'American',
 'football',
 'game',
 'to',
 'determine',
 'the',
 'champion',
 'of',
 'the',
 'National',
 'Football',
 'League',
 '(',
 'NFL',
 ')',
 'for',
 'the',
 '2015',
 'season',
 '.',
 'The',
 'American',
 'Football',
 'Conference',
 '(',
 'AFC',
 ')',
 'champion',
 'Denver',
 'Broncos',
 'defeated',
 'the',
 'National',
 'Football',
 'Conference',
 '(',
 'NFC',
 ')',
 'champion',
 'Carolina',
 'Panthers',
 '24–10',
 'to',
 'earn',
 'their',
 'third',
 'Super',
 'Bowl',
 'title',
 '.',
 'The',
 'game',
 'was',
 'played',
 'on',
 'February',
 '7',
 ',',
 '2016',
 ',',
 'at',
 'Levi',
 "'s",
 'Stadium',
 'in',
 'the',
 'San',
 'Francisco',
 'Bay',
 'Area',
 'at',
 'Santa',
 'Clara',
 ',',
 'California',
 '.',
 'As',
 'this',
 'was',
 'the',
 '50th',
 'Super',
 'Bowl',
 ',',
 'the',
 'league',
 'emphasized',
 'the',
 '"',
 'golden',
 'anniversary',
 '"',
 'with',
 'various',
 'gold-themed',
 'initiatives',
 ',',
 'as',
 'well',
 'as',
 'temp

In [25]:
par.context[33]

'Denver'

In [26]:
len(par.context)

147

In [27]:
qa = par.qas[0]

In [28]:
qa.question

['Which',
 'NFL',
 'team',
 'represented',
 'the',
 'AFC',
 'at',
 'Super',
 'Bowl',
 '50',
 '?']

In [29]:
a = qa.answers[0]

In [30]:
a.start_char

177

In [31]:
a.text

['Denver', 'Broncos']

In [32]:
a.start_token

33

In [33]:
a.end_token

35

## test get_word_embeddings

In [34]:
matrix, word_to_ind = tools.get_word_embeddings(c)

In [35]:
matrix.shape

(100000, 200)

In [36]:
matrix[:3]

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        

## test vectorize_paragraphs

In [37]:
padded_paragraphs = tools.vectorize_paragraphs(tokenized_paragraphs, word_to_ind, c)

WARNING! Provided context is too long. It will be cutted. 576 > 400
WARNING! Provided context is too long. It will be cutted. 525 > 400
WARNING! Provided context is too long. It will be cutted. 420 > 400
WARNING! Provided context is too long. It will be cutted. 469 > 400
WARNING! Provided context is too long. It will be cutted. 425 > 400
WARNING! Provided context is too long. It will be cutted. 567 > 400
WARNING! Provided context is too long. It will be cutted. 562 > 400
WARNING! Provided context is too long. It will be cutted. 441 > 400
WARNING! Provided context is too long. It will be cutted. 406 > 400
WARNING! Provided context is too long. It will be cutted. 616 > 400
WARNING! Provided context is too long. It will be cutted. 409 > 400
WARNING! Provided context is too long. It will be cutted. 700 > 400
WARNING! Provided context is too long. It will be cutted. 492 > 400
WARNING! Provided anser is too long. It will be cutted. 37 > 30
WARNING! Provided context is too long. It will be cu

In [38]:
len(padded_paragraphs)

2067

In [39]:
par = padded_paragraphs[0]

In [40]:
par.context.shape

(400,)

In [41]:
par.context

array([    0,     0,   693,    16,    30,     0,   536,   187,     5,
        2388,     1,  1060,     4,     1,     0,     0,     0,    24,
           0,    25,    11,     1,  9471,   196,     3,     0,     0,
           0,     0,    24,     0,    25,  1060,     0,     0,  1954,
           1,     0,     0,     0,    24,     0,    25,  1060,     0,
           0,     0,     5,  3850,    45,   246,     0,     0,   699,
           3,     0,   187,    16,   335,    14,     0,   633,     2,
       15464,     2,    23,     0,    10,     0,     7,     1,     0,
           0,     0,     0,    23,     0,     0,     2,     0,     3,
           0,    38,    16,     1,  7967,     0,     0,     2,     1,
         293,  6704,     1,     9,  2411,  2350,     9,    18,   918,
           0,  5724,     2,    20,   144,    20,  5054, 14484,     1,
        2837,     4,  7434,   237,     0,     0,   187,    18,     0,
       34138,    24,   125,    43,     1,   187,    55,    34,    52,
         226,    20,

In [42]:
word_to_ind['#']

2750

In [43]:
word_to_ind['#not_a_word#']

0

In [44]:
ind_to_word = {v:k for k, v in word_to_ind.items()}

In [45]:
[ind_to_word[ind] for ind in par.context]

['#not_a_word#',
 '#not_a_word#',
 '50',
 'was',
 'an',
 '#not_a_word#',
 'football',
 'game',
 'to',
 'determine',
 'the',
 'champion',
 'of',
 'the',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '(',
 '#not_a_word#',
 ')',
 'for',
 'the',
 '2015',
 'season',
 '.',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '(',
 '#not_a_word#',
 ')',
 'champion',
 '#not_a_word#',
 '#not_a_word#',
 'defeated',
 'the',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '(',
 '#not_a_word#',
 ')',
 'champion',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 'to',
 'earn',
 'their',
 'third',
 '#not_a_word#',
 '#not_a_word#',
 'title',
 '.',
 '#not_a_word#',
 'game',
 'was',
 'played',
 'on',
 '#not_a_word#',
 '7',
 ',',
 '2016',
 ',',
 'at',
 '#not_a_word#',
 "'s",
 '#not_a_word#',
 'in',
 'the',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 'at',
 '#not_a_word#',
 '#not_a_word#',
 ',',
 '#not_a_word#',
 '.',
 '#not_a_word#',
 'this',
 'was',


In [46]:
qa = par.qas[0]

In [47]:
qa.question.shape

(40,)

In [48]:
[ind_to_word[ind] for ind in qa.question]

['#not_a_word#',
 '#not_a_word#',
 'team',
 'represented',
 'the',
 '#not_a_word#',
 'at',
 '#not_a_word#',
 '#not_a_word#',
 '50',
 '?',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#']

In [49]:
a = qa.answers[0]

In [50]:
[ind_to_word[ind] for ind in a.text]

['#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#',
 '#not_a_word#']

In [51]:
a.start_token

33

In [52]:
a.end_token

35

In [53]:
a.start_char

177

## Test processed SQuAD

In [3]:
with open('SQuAD/meta.msgpack', 'rb') as f:
    meta = msgpack.load(f, encoding='utf8')

In [4]:
vocab = meta['vocab']

In [5]:
len(vocab)

91187

In [6]:
vocab_tag = meta['vocab_tag']
print(vocab_tag)

['NN', 'IN', 'NNP', 'DT', 'JJ', 'NNS', ',', '.', 'CC', 'VBD', 'RB', 'VBN', 'CD', 'VB', 'VBZ', 'VBG', 'PRP', 'VBP', 'TO', '-RRB-', 'HYPH', '-LRB-', 'PRP$', 'POS', 'WDT', "''", '``', 'MD', 'NNPS', ':', 'JJR', 'JJS', 'WRB', 'WP', 'RP', 'RBR', 'RBS', 'EX', '$', 'SYM', 'FW', 'NFP', 'AFX', 'PDT', 'WP$', 'UH', 'LS', 'SP', 'XX', 'ADD']


In [7]:
vocab_ent = meta['vocab_ent']

In [8]:
vocab_ent

['',
 'ORG',
 'DATE',
 'PERSON',
 'GPE',
 'CARDINAL',
 'NORP',
 'LOC',
 'WORK_OF_ART',
 'PERCENT',
 'EVENT',
 'ORDINAL',
 'MONEY',
 'FAC',
 'QUANTITY',
 'LAW',
 'TIME',
 'LANGUAGE',
 'PRODUCT']

In [9]:
embedding = meta['embedding']

In [10]:
len(embedding[0])

300

In [12]:
vocab[:30]

['<PAD>',
 '<UNK>',
 '?',
 'the',
 'What',
 'of',
 'in',
 'to',
 'was',
 'is',
 'did',
 'what',
 'a',
 "'s",
 'Who',
 'How',
 'for',
 'and',
 ',',
 'are',
 'many',
 'When',
 'that',
 'does',
 'were',
 '-',
 'on',
 'In',
 'with',
 'Which']

In [4]:
with open('SQuAD1/data.msgpack', 'rb') as f:
    data = msgpack.load(f, encoding='utf8')

In [5]:
train = data['train']

In [32]:
print('id', train[0][0])

id 5733be284776f41900661182


In [34]:
print('context_id', train[0][1])

context_id [56657, 18, 3, 137, 47, 12, 523, 991, 217, 77428, 3, 5543, 2424, 13, 1663, 10606, 9, 12, 6308, 4781, 5, 3, 3804, 711, 217, 36580, 6, 2094, 5, 3, 5543, 2424, 17, 6353, 50, 18, 9, 12, 753, 4781, 5, 2314, 28, 3053, 62876, 28, 3, 5798, 39, 78972, 15476, 8026, 66905, 39, 217, 22347, 7, 3, 5543, 2424, 9, 3, 4548, 5, 3, 10317, 4877, 217, 36580, 1573, 3, 10501, 9, 3, 22472, 18, 12, 12197, 122, 5, 5235, 17, 9189, 217, 2285, 9, 12, 12081, 5, 3, 71965, 45, 23146, 18, 197, 110, 3, 3804, 711, 44002, 1935, 7, 641, 48147, 48281, 6, 9045, 217, 216, 3, 140, 5, 3, 146, 2042, 916, 17, 6, 12, 1512, 393, 22, 2959, 244, 798, 6594, 17, 3, 4691, 12943, 828, 18, 9, 12, 2731, 18, 205, 2728, 4781, 5, 711, 217]


In [35]:
[vocab[id_] for id_ in train[0][1]]

['Architecturally',
 ',',
 'the',
 'school',
 'has',
 'a',
 'Catholic',
 'character',
 '.',
 'Atop',
 'the',
 'Main',
 'Building',
 "'s",
 'gold',
 'dome',
 'is',
 'a',
 'golden',
 'statue',
 'of',
 'the',
 'Virgin',
 'Mary',
 '.',
 'Immediately',
 'in',
 'front',
 'of',
 'the',
 'Main',
 'Building',
 'and',
 'facing',
 'it',
 ',',
 'is',
 'a',
 'copper',
 'statue',
 'of',
 'Christ',
 'with',
 'arms',
 'upraised',
 'with',
 'the',
 'legend',
 '"',
 'Venite',
 'Ad',
 'Me',
 'Omnes',
 '"',
 '.',
 'Next',
 'to',
 'the',
 'Main',
 'Building',
 'is',
 'the',
 'Basilica',
 'of',
 'the',
 'Sacred',
 'Heart',
 '.',
 'Immediately',
 'behind',
 'the',
 'basilica',
 'is',
 'the',
 'Grotto',
 ',',
 'a',
 'Marian',
 'place',
 'of',
 'prayer',
 'and',
 'reflection',
 '.',
 'It',
 'is',
 'a',
 'replica',
 'of',
 'the',
 'grotto',
 'at',
 'Lourdes',
 ',',
 'France',
 'where',
 'the',
 'Virgin',
 'Mary',
 'reputedly',
 'appeared',
 'to',
 'Saint',
 'Bernadette',
 'Soubirous',
 'in',
 '1858',
 '.',
 'At

In [36]:
print('context_features', train[0][2])

context_features [[False, False, False, 0.007042253521126761], [False, False, False, 0.04225352112676056], [True, True, True, 0.1056338028169014], [False, False, False, 0.007042253521126761], [False, False, False, 0.007042253521126761], [False, False, False, 0.04929577464788732], [False, False, False, 0.007042253521126761], [False, False, False, 0.007042253521126761], [False, False, False, 0.04929577464788732], [False, False, False, 0.007042253521126761], [True, True, True, 0.1056338028169014], [False, False, False, 0.028169014084507043], [False, False, False, 0.02112676056338028], [False, False, False, 0.007042253521126761], [False, False, False, 0.014084507042253521], [False, False, False, 0.014084507042253521], [False, False, False, 0.04225352112676056], [False, False, False, 0.04929577464788732], [False, False, False, 0.007042253521126761], [False, False, False, 0.02112676056338028], [False, False, False, 0.056338028169014086], [True, True, True, 0.1056338028169014], [True, True, T

In [37]:
print('tag_id', train[0][3])

tag_id [10, 6, 3, 0, 14, 3, 4, 0, 7, 1, 3, 2, 2, 23, 0, 0, 14, 3, 4, 0, 1, 3, 2, 2, 7, 10, 1, 0, 1, 3, 2, 2, 8, 15, 16, 6, 14, 3, 0, 0, 1, 2, 1, 5, 4, 1, 3, 0, 26, 2, 2, 16, 2, 25, 7, 10, 1, 3, 2, 2, 14, 3, 2, 1, 3, 2, 2, 7, 10, 1, 3, 0, 14, 3, 2, 6, 3, 2, 0, 1, 0, 8, 0, 7, 16, 14, 3, 0, 1, 3, 0, 1, 2, 6, 2, 32, 3, 2, 2, 10, 9, 1, 2, 2, 2, 1, 12, 7, 1, 3, 0, 1, 3, 4, 0, 21, 8, 1, 3, 4, 0, 24, 14, 1, 12, 5, 8, 3, 2, 2, 19, 6, 14, 3, 4, 6, 4, 0, 0, 1, 2, 7]


In [53]:
len(vocab_tag)

50

In [51]:
[vocab_tag[id_] for id_ in train[0][3]]

['RB',
 ',',
 'DT',
 'NN',
 'VBZ',
 'DT',
 'JJ',
 'NN',
 '.',
 'IN',
 'DT',
 'NNP',
 'NNP',
 'POS',
 'NN',
 'NN',
 'VBZ',
 'DT',
 'JJ',
 'NN',
 'IN',
 'DT',
 'NNP',
 'NNP',
 '.',
 'RB',
 'IN',
 'NN',
 'IN',
 'DT',
 'NNP',
 'NNP',
 'CC',
 'VBG',
 'PRP',
 ',',
 'VBZ',
 'DT',
 'NN',
 'NN',
 'IN',
 'NNP',
 'IN',
 'NNS',
 'JJ',
 'IN',
 'DT',
 'NN',
 '``',
 'NNP',
 'NNP',
 'PRP',
 'NNP',
 "''",
 '.',
 'RB',
 'IN',
 'DT',
 'NNP',
 'NNP',
 'VBZ',
 'DT',
 'NNP',
 'IN',
 'DT',
 'NNP',
 'NNP',
 '.',
 'RB',
 'IN',
 'DT',
 'NN',
 'VBZ',
 'DT',
 'NNP',
 ',',
 'DT',
 'NNP',
 'NN',
 'IN',
 'NN',
 'CC',
 'NN',
 '.',
 'PRP',
 'VBZ',
 'DT',
 'NN',
 'IN',
 'DT',
 'NN',
 'IN',
 'NNP',
 ',',
 'NNP',
 'WRB',
 'DT',
 'NNP',
 'NNP',
 'RB',
 'VBD',
 'IN',
 'NNP',
 'NNP',
 'NNP',
 'IN',
 'CD',
 '.',
 'IN',
 'DT',
 'NN',
 'IN',
 'DT',
 'JJ',
 'NN',
 '-LRB-',
 'CC',
 'IN',
 'DT',
 'JJ',
 'NN',
 'WDT',
 'VBZ',
 'IN',
 'CD',
 'NNS',
 'CC',
 'DT',
 'NNP',
 'NNP',
 '-RRB-',
 ',',
 'VBZ',
 'DT',
 'JJ',
 ',',
 'JJ',
 'N

In [40]:
print('ent_id', train[0][4])

ent_id [0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 13, 13, 13, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 13, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 8, 8, 8, 8, 0, 0, 0, 0, 13, 13, 13, 0, 0, 3, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 3, 0, 0, 0, 3, 3, 3, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0]


In [54]:
len(vocab_ent)

19

In [52]:
[vocab_ent[id_] for id_ in train[0][4]]

['',
 '',
 '',
 '',
 '',
 '',
 'NORP',
 '',
 '',
 '',
 'FAC',
 'FAC',
 'FAC',
 'FAC',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'FAC',
 'FAC',
 'FAC',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'NORP',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'WORK_OF_ART',
 'WORK_OF_ART',
 'WORK_OF_ART',
 'WORK_OF_ART',
 '',
 '',
 '',
 '',
 'FAC',
 'FAC',
 'FAC',
 '',
 '',
 'PERSON',
 '',
 'ORG',
 'ORG',
 'ORG',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'GPE',
 '',
 '',
 'PERSON',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'GPE',
 '',
 'GPE',
 '',
 '',
 '',
 'PERSON',
 '',
 '',
 '',
 'PERSON',
 'PERSON',
 'PERSON',
 '',
 'DATE',
 '',
 '',
 'DATE',
 'DATE',
 'DATE',
 'DATE',
 'DATE',
 'DATE',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'CARDINAL',
 '',
 '',
 '',
 '',
 'PERSON',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'PERSON',
 '']

In [43]:
print('question_id', train[0][5])

question_id [224, 215, 10, 3, 3804, 711, 5858, 627, 6, 9045, 6, 23146, 197, 2]


In [55]:
[vocab[id_] for id_ in train[0][5]]

['To',
 'whom',
 'did',
 'the',
 'Virgin',
 'Mary',
 'allegedly',
 'appear',
 'in',
 '1858',
 'in',
 'Lourdes',
 'France',
 '?']

In [46]:
print('context', train[0][6])

context Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.


In [47]:
print('context_token_span', train[0][7])

context_token_span [[0, 15], [15, 16], [17, 20], [21, 27], [28, 31], [32, 33], [34, 42], [43, 52], [52, 53], [54, 58], [59, 62], [63, 67], [68, 76], [76, 78], [79, 83], [84, 88], [89, 91], [92, 93], [94, 100], [101, 107], [108, 110], [111, 114], [115, 121], [122, 126], [126, 127], [128, 139], [140, 142], [143, 148], [149, 151], [152, 155], [156, 160], [161, 169], [170, 173], [174, 180], [181, 183], [183, 184], [185, 187], [188, 189], [190, 196], [197, 203], [204, 206], [207, 213], [214, 218], [219, 223], [224, 232], [233, 237], [238, 241], [242, 248], [249, 250], [250, 256], [257, 259], [260, 262], [263, 268], [268, 269], [269, 270], [271, 275], [276, 278], [279, 282], [283, 287], [288, 296], [297, 299], [300, 303], [304, 312], [313, 315], [316, 319], [320, 326], [327, 332], [332, 333], [334, 345], [346, 352], [353, 356], [357, 365], [366, 368], [369, 372], [373, 379], [379, 380], [381, 382], [383, 389], [390, 395], [396, 398], [399, 405], [406, 409], [410, 420], [420, 421], [422, 424]

In [49]:
print('answer_start', train[0][8])

answer_start 102


In [50]:
print('answer_end', train[0][9])

answer_end 104


In [6]:
b = tools.reader(train[0])

In [8]:
[print(i.shape) for i in b[0]]

(40,)
(400,)
(400,)
(400,)
(400, 262)
()
()


[None, None, None, None, None, None, None]

In [17]:
x = np.array(train[0][2])

In [18]:
x.shape

(142, 4)

In [20]:
import numpy as np

In [22]:
padded = np.pad(x, [(0, 400-142),(0,0)],  'constant', constant_values = -1)

In [23]:
padded.shape

(400, 4)

In [25]:
x

array([[ 0.        ,  0.        ,  0.        ,  0.00704225],
       [ 0.        ,  0.        ,  0.        ,  0.04225352],
       [ 1.        ,  1.        ,  1.        ,  0.1056338 ],
       [ 0.        ,  0.        ,  0.        ,  0.00704225],
       [ 0.        ,  0.        ,  0.        ,  0.00704225],
       [ 0.        ,  0.        ,  0.        ,  0.04929577],
       [ 0.        ,  0.        ,  0.        ,  0.00704225],
       [ 0.        ,  0.        ,  0.        ,  0.00704225],
       [ 0.        ,  0.        ,  0.        ,  0.04929577],
       [ 0.        ,  0.        ,  0.        ,  0.00704225],
       [ 1.        ,  1.        ,  1.        ,  0.1056338 ],
       [ 0.        ,  0.        ,  0.        ,  0.02816901],
       [ 0.        ,  0.        ,  0.        ,  0.02112676],
       [ 0.        ,  0.        ,  0.        ,  0.00704225],
       [ 0.        ,  0.        ,  0.        ,  0.01408451],
       [ 0.        ,  0.        ,  0.        ,  0.01408451],
       [ 0.        ,  0.